In [289]:
import pyvisa
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib
import time

In [290]:
amplitude=20
Group='A'
path='C:\\Users\\Administrator\\Desktop\\RIGOL\\Demo_python\\GroupB'

In [291]:
time_interval = 500e-6  #示波器横坐标时间间隔,单位s
voltage =2    #电压范围,单位V，python图像显示范围
x_left = -time_interval*0.5       #横坐标起点
x_right = time_interval*10         #横坐标终点
y_up = voltage*amplitude*1e-1     #纵坐标上限
y_down = -voltage*amplitude*1e-1  #纵坐标下限
y_scale=500     #垂直刻度，单位mV，示波器纵轴间隔
x_position=2e-3     #设置触发位置，单位s
time_pause=10    #暂停时间，单位S
Average=32  #平均次数

# 确保使用正确的图形后
matplotlib.use('TkAgg')

# 打开示波器
# 创建ResourceManager对象
rm_scope = pyvisa.ResourceManager()

# 连接示波器（请替换为你的实际设备地址）
scope_address = 'TCPIP::192.168.1.101::INSTR'  # 地址
scope = rm_scope.open_resource(scope_address)

# 获取示波器ID信息，确认连接成功
idn = scope.query('*IDN?')
scope.write(':RUN')

trigger_level = amplitude*1e-3  # 设定触发电平单位S
scope.write(f':TRIGger:SHOLd:DLEVel {trigger_level}')

# 配置示波器参数（可根据需求修改）
scope.write(':CHANNEL1:VOLTAGE:OFFSET 0')               # 设置电压偏移
scope.write(f':CHANNEL1:VOLTAGE:SCALE {voltage}')       # 设置电压间隔
scope.write(f':TIMEBASE:SCALE {time_interval}')         # 设置时间间隔
scope.write(f':CHANnel1:SCALe {y_scale}mv')             # 设置y轴刻度
scope.write(f':TIMebase:MAIN:OFFSet {x_position}')      #设置触发位置，单位s
scope.write(':ACQuire:TYPE AVERages')      #平均模式
scope.write(f':ACQuire:AVERages {Average}') #平均模式32次


# 获取波形数据的函数
def get_waveform():
    # 设置读取数据点      
    scope.write(':WAVEFORM:FORMAT ASCII')         
    scope.write(':WAVEFORM:SOURCE CHANNEL1')      

    # 读取波形数据
    data = scope.query(':WAVEFORM:DATA?')

    # 去除前缀并清洗数据
    if data.startswith('#'):
        header_length = int(data[1])  # 获取长度标识符
        offset = 2 + header_length     # 计算数据起始位置
        data = data[offset:]           # 保留有效波形数据部分

    # 将数据转换为浮动类型
    data_list = [item for item in data.split(',') if item.strip()]  # 清理空值
    waveform_data = np.array(data_list, dtype=float)

    # 获取X轴的时间增量和起始时间
    x_increment = float(scope.query(':WAVEFORM:XINCREMENT?'))  # 时间增量
    x_origin = float(scope.query(':WAVEFORM:XORIGIN?'))        # 起始时间
    x_values = x_origin + np.arange(len(waveform_data)) * x_increment  # 计算X轴数据
    return x_values, waveform_data

# 创建图形
fig, ax = plt.subplots(figsize=(10, 6))
line, = ax.plot([], [], lw=2)

# 设置图形的标题和标签
ax.set_title(f"Real-Time Oscilloscope Waveform for {amplitude}VPP")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Voltage (V)")
ax.grid(True)

# 设置横坐标和纵坐标范围
ax.set_xlim([x_left, x_right])  #设置时间范围
ax.set_ylim([y_down, y_up])    # 设置电压范围

# 初始化图形（空数据）
def init():
    line.set_data([], [])
    return line,

# 更新图形的数据
def update(frame):
    x_values, waveform_data = get_waveform()
    line.set_data(x_values, waveform_data)
    return line,

# 创建动画
ani = animation.FuncAnimation(fig, update, frames=100, init_func=init, blit=True, interval=50)

# 显示图形
plt.show()
scope.write(":TRIGger:SWEep SINGle")  #设置触发模式为“单次” 
Save_depth=100
scope.write(f':ACQuire:MDEPth {Save_depth}k')
time.sleep(1)  

In [ ]:
# 初始化与示波器的连接
rm = pyvisa.ResourceManager()
scope = rm.open_resource('TCPIP::192.168.1.101::INSTR')  # 替换为实际设备地址

# 设置波形参数
scope.write(':WAV:MODE MAXimum')  # 设置波形模式
scope.write(':WAV:FORM BYTE')  # 设置波形格式为字节
scope.write(f':WAVeform:POINts {Save_depth}k')  # 设置采样点数

# 获取波形参数
x_increment = float(scope.query(':WAV:XINC?'))  # 时间分辨率
x_origin = float(scope.query(':WAV:XOR?'))  # 时间起点
y_increment = float(scope.query(':WAV:YINC?'))  # 电压分辨率
y_origin = float(scope.query(':WAV:YOR?'))  # 电压起点
y_reference = float(scope.query(':WAV:YREF?'))  # 电压参考点

# 读取波形数据（原始值为无符号字节）
raw_data = scope.query_binary_values(':WAV:DATA?', datatype='B', container=np.array)
scope.write(":TRIGger:SWEep SINGle")  #设置触发模式为“单次” 
time.sleep(1)

# 转换波形数据为物理值
voltages = (raw_data - y_reference) * y_increment + y_origin
times = np.arange(0, len(voltages)) * x_increment + x_origin
print(len(voltages))
# 检查转换后的最小值和最大值
print(f"Voltage range: {voltages.min()} V to {voltages.max()} V")

# 保存数据到 CSV 文件
output_file = f'{path}\\{Group}{amplitude}.csv'
np.savetxt(output_file, np.column_stack((times, voltages)), delimiter=',', header='Time(s),Voltage(V)', comments='')
print(f"波形数据已保存到 {output_file}")

In [293]:
import pandas as pd
import matplotlib.pyplot as plt

# 读取CSV文件
csv_file_path = f'{path}\\{Group}{amplitude}.csv'  # 替换为你的CSV文件路径
data = pd.read_csv(csv_file_path)

# 假设CSV文件的第一列名称为'Column1'，如果不同请替换为你的列名
column_name = 'Voltage(V)'

# 检查列名是否存在
if column_name in data.columns:
    # 读取第一列数据
    column_data = data[column_name]
    
    # 绘制第一列数据
    plt.figure(figsize=(10, 5))  # 设置图形大小
    plt.plot(column_data, label=column_name)  # 绘制线图
    plt.title('Plot of ' + column_name)  # 设置图形标题
    plt.xlabel('Index')  # 设置x轴标签
    plt.ylabel(column_name)  # 设置y轴标签
    plt.legend()  # 显示图例
    plt.grid(True)  # 显示网格
    plt.show()  # 显示图形
else:
    print(f"Column '{column_name}' does not exist in the CSV file.")